In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

In [5]:
df = pd.read_excel('data.xls')

In [6]:
def change(val):
    val2 = val.replace(' ', '_')
    return val2

In [7]:
df['Names Corrected'] = df['Names'].apply(change)
df

,Names,Names Corrected
0,Artificial Intelligence,Artificial_Intelligence
1,Blockchain,Blockchain
2,Internet of things,Internet_of_things
3,Cloud Computing,Cloud_Computing
4,Amazon,Amazon
5,Laptop,Laptop
6,Mobile,Mobile
7,Computer,Computer
8,Windows,Windows
9,Steve Jobs,Steve_Jobs


In [8]:
names = df["Names Corrected"].to_list()

In [56]:
source = []
head1  = []
head2  = []
head3  = []
para   = []
count  = 0
csv_row = []

for i in names:
    data = requests.get(f'https://en.wikipedia.org/wiki/{i}')
    soup = BeautifulSoup(data.content, 'html.parser')
    source.append(i)
    body = soup.body    
    paragraph = body.find_all('p')   
    for p in paragraph:
        text = p.text.replace("\n", "")
        if text != "":
            para.append(text)            
   
    header1 = body.find_all('h1')
    for p in header1:
        text = p.text.replace("\n", "")
        if text != "":
            head1.append(text)
    header2 = body.find_all('h2')
    for p in header2:
        text = p.text.replace("\n", "")
        if text != "":
            head2.append(text)
    header3 = body.find_all('h3')
    for p in header3:
        text = p.text.replace("\n", "")
        if text != "":
            head3.append(text)
    count += 1
    
#all_data = {"Heading 1": head1, "Heading 2": head2, "Heading 3": head3, "Paragraph": para}
#cf = pd.DataFrame(all_data)
#cf.to_csv('export_html.csv')

max_count = max(list([len(head1),len(head2),len(head3),len(para)]))

# Step 2: Export the data to csv
"""
For this example let's create 3 seperate csv for 
3 tables respectively
"""
# Create csv file for each table
with open(f"export_html.csv", 'w', encoding="utf-8", newline='') as out_file:    
    # Each 3 table has headers as following
    headers = [         
        "Heading 1",
        "Heading 2",
        "Heading 3",
        "Paragraph"
    ] # == t_headers
    writer = csv.DictWriter(out_file, headers)
    # write the header
    writer.writeheader()
    
    j = 0
    
    for i in range(max_count):        
        h1 = head1[i] if i < len(head1) else ""
        h2 = head2[i] if i < len(head2) else ""
        h3 = head3[i] if i < len(head3) else ""
        p1 = para[i] if i < len(para) else ""
        write_data = {'Heading 1': h1, 'Heading 2': h2, 'Heading 3': h3, 'Paragraph': p1}
        #print("{},{},{},{}".format(h1, h2, h3, p1))
        writer.writerow(write_data)
        